> {sub-ref}`today` | {sub-ref}`wordcount-minutes` min read

(sec_Notebooks_oraculos)= 
# Oráculos (funciones digitales)


- **{ref}`sec_Notebooks_oraculos_1`**
- **{ref}`sec_Notebooks_oraculos_2`**

In [ ]:
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit import transpile
from qiskit.circuit.library import MCXGate
import numpy as np
from qiskit_aer import AerSimulator

In [ ]:
# Importamos el simulador. Con "method" le especificamos el método de simulación
simulador = AerSimulator(method = 'statevector')

(sec_Notebooks_oraculos_1)= 
##  Construcción de funciones binarias. Los min-términos 

Consideremos la siguiente tabla de verdad para una función $f: \{0, 1\}^3 \rightarrow \{0, 1\}$ concreta.


|$$x_2$$|$$x_1$$|$$x_0$$|$$f(x)$$|
|---|---|---|---|
|0|0|0|0|
|0|0|1|1|
|0|1|0|0|
|0|1|1|0|
|1|0|0|0|
|1|0|1|1|
|1|1|0|0|
|1|1|1|1|

La idea es considerar exclusivamente los términos que tienen como salida la variable 1, que denominaremos <b>min-términos</b>. 

Cada min-término llevará asociada una puerta condicionada diferente. Su composición define la función $f$

Para el caso de la tabla de verdad anterior, el circuito correspondiente vendrá dado por:

In [ ]:
qr = QuantumRegister(4)
cr = ClassicalRegister(4)

qc = QuantumCircuit(qr, cr, name='q')

qc.append(MCXGate(3, ctrl_state=1), qr)
qc.append(MCXGate(3, ctrl_state=5), qr)
qc.append(MCXGate(3, ctrl_state=7), qr)

qc.draw(output='mpl')

donde hemos hecho uso de la puerta multicontrolada [MCXGate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.MCXGate.html?highlight=mcxgate#qiskit.circuit.library.MCXGate) de qiskit

Vamos a implementar una función $f:\{0,1\}^4\to \{0,1\}^4$ dada por la siguiente *tabla de verdad* 


|$$x$$|$$f(x)$$|$$x$$|$$f(x)$$|
|---|---|---|---|
|0000|1111|1000|0101|
|0001|1011|1001|0100|
|0010|0011|1010|0000|
|0011|1000|1011|1110|
|0100|0101|1100|1111|
|0101|0100|1101|1011|
|0110|0000|1110|0011|
|0111|1110|1111|1000|



::::::{admonition} Ejercicio
:class: tip


        
Completa la el código que genera un circuito que implementa la siguiente función digital
 

:::{dropdown} Solución
    if output_bit =='1':
        qc.append(MCXGate(len(input_str), ctrl_state=ctrl_state),qr_input[:]+[qr_output[j]])
:::
::::::


In [ ]:
def oracle(f_outputs): 
    
    n = int(np.log2(len(f_outputs)))  #dimension del registro de entrada |x> 
    m = len(f_outputs[0])             #dimension del registro de salida |f(x)>
    
    #generamos todos los posibles inputs en binario, completando con ceros hasta tener strings de n bits
    inputs = [format(i, 'b').zfill(n) for i in range(2**n)]
    print(inputs)
    
    qr_input = QuantumRegister(n, name='input')
    qr_output = QuantumRegister(m, name='output')
    qc = QuantumCircuit(qr_input, qr_output)


    # Hacemos un bucle sobre los inputs
    for i,input_str in enumerate(inputs):
        ctrl_state= int(input_str[::],2)
        print(i, ctrl_state)
        # Para cada input, i, haz un bucle sobre cada bit del output     
        for j,output_bit in enumerate(f_outputs[i]):
        
###
#
#        Tu solución aquí
#
#
####
    return qc

In [ ]:
f_outputs = ['1111', '1011', '0011', '1000', '0101', '0100', 
               '0000', '1110', '0101', '0100', '0000', '1110', 
               '1111', '1011', '0011', '1000']

# f_outputs= ['000', '001', '010', '011', '100', '101', '110', '111']
    
   
oracle(f_outputs).draw()

::::::{admonition} Ejercicio
:class: tip


        
Escribe una función $f:S^n\to S$  que  produzca aleatoriamente $f(x) = \pm 1$ de forma <i>equilibrada</i> (es decir, tantos $f(x)= +1$ como $f(x)= -1$). 
 

:::{dropdown} Solución

Ver por ejemplo la solución del qiskit textbook:https://qiskit.org/textbook/ch-algorithms/deutsch-jozsa.html   sección 4.4 
    
:::
::::::


(sec_Notebooks_oraculos_2)= 
## Función binaria lineal
 

Dados dos n-tuplas binarias $x=(x_{n-1},\ldots,x_0)$ y $a=(a_{n-1},\ldots,a_0)$ definimos la **función lineal**

        
\begin{equation}
f(x;a) = a \cdot x = a_{n-1} x_{n-1} \oplus a_{n-2} x_{n-2} \oplus \cdots \oplus a_{0} x_{0}\; ,
\end{equation}


donde  $\oplus$ es la suma módulo 2.

Por ejemplo, el circuito que implementa esta función cuando $a=11010$ es el siguiente




::::{figure} Figuras/Fig_InitialOracle_linear_function.png
:width: 400px
:align: center
::::


::::::{admonition} Ejercicio
:class: tip


        
Completa el siguiente código que genera el circuito asociado a la función binaria lineal $f(x;a)$. 
 

:::{dropdown} Solución

    for i, aq in enumerate(reversed(a)):
        if aq == '1':
             qc.cx(qr_in[i],qr_out) 
    
:::
::::::


```{code-block} python
def linear_circuit(x,a):
       
    assert(len(x)==len(a))

    # Inicialización de los registros
    qr_in = QuantumRegister(len(a), name='qr_in')
    qr_out = QuantumRegister(1, name='qr_out')
    cr = ClassicalRegister(1, name='cr')  
    qc = QuantumCircuit(qr_in, qr_out, cr, name='q_linear')
    
    'inicializamo el estado x '
    for i, xq in enumerate(reversed(x)):  # ojo con la ordenación de qiskit, por eso está reversed()
        if xq == '1':
             qc.x(qr_in[i]) 

    qc.barrier()

    'codificamos la función lineal x.a '
###
#
#        Tu solución aquí
#
#
####
            
    qc.barrier()
    qc.measure(qr_out[0],cr[0])
    
    return qc 
```


Veamos un ejemplo

```{code-block} python
a = '1011'
x = '1001'

circuit = linear_circuit(x,a)
#qc.draw('mpl')
```


La función $a\cdot x = (1 + 0 + 0 + 1)mod(2) = 0$. Vamos a ver si este resultado es el hallado

```{code-block} python
# transpilamos
t_circuit = transpile(circuit, backend = simulador)

# Ejecutamos la simulación con 1000 shots 
result = simulador.run(t_circuit, shots = 1000).result()
counts = result.get_counts()
counts
```
